# Experiments using DDPM for OOD

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List, Dict, Optional, Tuple, Union
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, FashionMNIST
from torch.utils.data import DataLoader
import random
import math

##DDPM model

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class TimeEmbedding(nn.Module):
    """
    ### Embeddings for $t$
    """

    def __init__(self, n_channels: int):
        """
        * `n_channels` is the number of dimensions in the embedding
        """
        super().__init__()
        self.n_channels = n_channels
        # First linear layer
        self.lin1 = nn.Linear(self.n_channels // 4, self.n_channels)
        # Activation
        self.act = nn.SiLU()
        # Second linear layer
        self.lin2 = nn.Linear(self.n_channels, self.n_channels)

    def forward(self, t: torch.Tensor):
        # Create sinusoidal position embeddings
        # [same as those from the transformer](../../transformers/positional_encoding.html)
        #
        # \begin{align}
        # PE^{(1)}_{t,i} &= sin\Bigg(\frac{t}{10000^{\frac{i}{d - 1}}}\Bigg) \\
        # PE^{(2)}_{t,i} &= cos\Bigg(\frac{t}{10000^{\frac{i}{d - 1}}}\Bigg)
        # \end{align}
        #
        # where $d$ is `half_dim`
        half_dim = self.n_channels // 8
        emb = math.log(10_000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=t.device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=1)

        # Transform with the MLP
        emb = self.act(self.lin1(emb))
        emb = self.lin2(emb)

        #
        return emb


class ResidualBlock(nn.Module):
    """
    ### Residual block

    A residual block has two convolution layers with group normalization.
    Each resolution is processed with two residual blocks.
    """

    def __init__(self, in_channels: int, out_channels: int, time_channels: int,
                 n_groups: int = 2, dropout: float = 0.5):
        """
        * `in_channels` is the number of input channels
        * `out_channels` is the number of input channels
        * `time_channels` is the number channels in the time step ($t$) embeddings
        * `n_groups` is the number of groups for [group normalization](../../normalization/group_norm/index.html)
        * `dropout` is the dropout rate
        """
        super().__init__()
        # Group normalization and the first convolution layer
        self.norm1 = nn.GroupNorm(n_groups, in_channels)
        self.act1 = nn.SiLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), padding=(1, 1))

        # Group normalization and the second convolution layer
        self.norm2 = nn.GroupNorm(n_groups, out_channels)
        self.act2 = nn.SiLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), padding=(1, 1))

        # If the number of input channels is not equal to the number of output channels we have to
        # project the shortcut connection
        if in_channels != out_channels:
            self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1))
        else:
            self.shortcut = nn.Identity()

        # Linear layer for time embeddings
        self.time_emb = nn.Linear(time_channels, out_channels)
        self.time_act = nn.SiLU()

        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        """
        * `x` has shape `[batch_size, in_channels, height, width]`
        * `t` has shape `[batch_size, time_channels]`
        """
        # First convolution layer
        h = self.conv1(self.act1(self.norm1(x)))
        # Add time embeddings
        h += self.time_emb(self.time_act(t))[:, :, None, None]
        # Second convolution layer
        h = self.conv2(self.dropout(self.act2(self.norm2(h))))

        # Add the shortcut connection and return
        return h + self.shortcut(x)

class AttentionBlock(nn.Module):
    """
    ### Attention block

    This is similar to [transformer multi-head attention](../../transformers/mha.html).
    """

    def __init__(self, n_channels: int, n_heads: int = 1, d_k: int = None, n_groups: int = 4):
        """
        * `n_channels` is the number of channels in the input
        * `n_heads` is the number of heads in multi-head attention
        * `d_k` is the number of dimensions in each head
        * `n_groups` is the number of groups for [group normalization](../../normalization/group_norm/index.html)
        """
        super().__init__()

        # Default `d_k`
        if d_k is None:
            d_k = n_channels
        # Normalization layer
        self.norm = nn.GroupNorm(n_groups, n_channels)
        # Projections for query, key and values
        self.projection = nn.Linear(n_channels, n_heads * d_k * 3)
        # Linear layer for final transformation
        self.output = nn.Linear(n_heads * d_k, n_channels)
        # Scale for dot-product attention
        self.scale = d_k ** -0.5
        #
        self.n_heads = n_heads
        self.d_k = d_k

    def forward(self, x: torch.Tensor, t: Optional[torch.Tensor] = None):
        """
        * `x` has shape `[batch_size, in_channels, height, width]`
        * `t` has shape `[batch_size, time_channels]`
        """
        # `t` is not used, but it's kept in the arguments because for the attention layer function signature
        # to match with `ResidualBlock`.
        _ = t
        # Get shape
        batch_size, n_channels, height, width = x.shape
        # Change `x` to shape `[batch_size, seq, n_channels]`
        x = x.view(batch_size, n_channels, -1).permute(0, 2, 1)
        # Get query, key, and values (concatenated) and shape it to `[batch_size, seq, n_heads, 3 * d_k]`
        qkv = self.projection(x).view(batch_size, -1, self.n_heads, 3 * self.d_k)
        # Split query, key, and values. Each of them will have shape `[batch_size, seq, n_heads, d_k]`
        q, k, v = torch.chunk(qkv, 3, dim=-1)
        # Calculate scaled dot-product $\frac{Q K^\top}{\sqrt{d_k}}$
        attn = torch.einsum('bihd,bjhd->bijh', q, k) * self.scale
        # Softmax along the sequence dimension $\underset{seq}{softmax}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)$
        attn = attn.softmax(dim=2)
        # Multiply by values
        res = torch.einsum('bijh,bjhd->bihd', attn, v)
        # Reshape to `[batch_size, seq, n_heads * d_k]`
        res = res.view(batch_size, -1, self.n_heads * self.d_k)
        # Transform to `[batch_size, seq, n_channels]`
        res = self.output(res)

        # Add skip connection
        res += x

        # Change to shape `[batch_size, in_channels, height, width]`
        res = res.permute(0, 2, 1).view(batch_size, n_channels, height, width)

        #
        return res


class DownBlock(nn.Module):
    """
    ### Down block

    This combines `ResidualBlock` and `AttentionBlock`. These are used in the first half of U-Net at each resolution.
    """

    def __init__(self, in_channels: int, out_channels: int, time_channels: int, has_attn: bool):
        super().__init__()
        self.res = ResidualBlock(in_channels, out_channels, time_channels)
        if has_attn:
            self.attn = AttentionBlock(out_channels)
        else:
            self.attn = nn.Identity()

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        x = self.res(x, t)
        x = self.attn(x)
        return x


class UpBlock(nn.Module):
    """
    ### Up block

    This combines `ResidualBlock` and `AttentionBlock`. These are used in the second half of U-Net at each resolution.
    """

    def __init__(self, in_channels: int, out_channels: int, time_channels: int, has_attn: bool):
        super().__init__()
        # The input has `in_channels + out_channels` because we concatenate the output of the same resolution
        # from the first half of the U-Net
        self.res = ResidualBlock(in_channels + out_channels, out_channels, time_channels)
        if has_attn:
            self.attn = AttentionBlock(out_channels)
        else:
            self.attn = nn.Identity()

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        x = self.res(x, t)
        x = self.attn(x)
        return x


class MiddleBlock(nn.Module):
    """
    ### Middle block

    It combines a `ResidualBlock`, `AttentionBlock`, followed by another `ResidualBlock`.
    This block is applied at the lowest resolution of the U-Net.
    """

    def __init__(self, n_channels: int, time_channels: int):
        super().__init__()
        self.res1 = ResidualBlock(n_channels, n_channels, time_channels)
        self.attn = AttentionBlock(n_channels)
        self.res2 = ResidualBlock(n_channels, n_channels, time_channels)

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        x = self.res1(x, t)
        x = self.attn(x)
        x = self.res2(x, t)
        return x


class Upsample(nn.Module):
    """
    ### Scale up the feature map by $2 \times$
    """

    def __init__(self, n_channels):
        super().__init__()
        self.conv = nn.ConvTranspose2d(n_channels, n_channels, (4, 4), (2, 2), (1, 1))

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        # `t` is not used, but it's kept in the arguments because for the attention layer function signature
        # to match with `ResidualBlock`.
        _ = t
        return self.conv(x)


class Downsample(nn.Module):
    """
    ### Scale down the feature map by $\frac{1}{2} \times$
    """

    def __init__(self, n_channels):
        super().__init__()
        self.conv = nn.Conv2d(n_channels, n_channels, (3, 3), (2, 2), (1, 1))

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        # `t` is not used, but it's kept in the arguments because for the attention layer function signature
        # to match with `ResidualBlock`.
        _ = t
        return self.conv(x)


class UNet(nn.Module):
    """
    ## U-Net
    """

    def __init__(self, image_channels: int = 1, n_channels: int = 16,
                 ch_mults: Union[Tuple[int, ...], List[int]] = (1, 2, 2, 4),
                 is_attn: Union[Tuple[bool, ...], List[bool]] = (False, False, True, True),
                 n_blocks: int = 2):
        """
        * `image_channels` is the number of channels in the image. $3$ for RGB.
        * `n_channels` is number of channels in the initial feature map that we transform the image into
        * `ch_mults` is the list of channel numbers at each resolution. The number of channels is `ch_mults[i] * n_channels`
        * `is_attn` is a list of booleans that indicate whether to use attention at each resolution
        * `n_blocks` is the number of `UpDownBlocks` at each resolution
        """
        super().__init__()

        # Number of resolutions
        n_resolutions = len(ch_mults)

        # Project image into feature map
        self.image_proj = nn.Conv2d(image_channels, n_channels, kernel_size=(3, 3), padding=(1, 1))

        # Time embedding layer. Time embedding has `n_channels * 4` channels
        self.time_emb = TimeEmbedding(n_channels * 4)

        # #### First half of U-Net - decreasing resolution
        down = []
        # Number of channels
        out_channels = in_channels = n_channels
        # For each resolution
        for i in range(n_resolutions):
            # Number of output channels at this resolution
            out_channels = in_channels * ch_mults[i]
            # Add `n_blocks`
            for _ in range(n_blocks):
                down.append(DownBlock(in_channels, out_channels, n_channels * 4, is_attn[i]))
                in_channels = out_channels
            # Down sample at all resolutions except the last
            if i < n_resolutions - 1:
                down.append(Downsample(in_channels))

        # Combine the set of modules
        self.down = nn.ModuleList(down)

        # Middle block
        self.middle = MiddleBlock(out_channels, n_channels * 4, )

        # #### Second half of U-Net - increasing resolution
        up = []
        # Number of channels
        in_channels = out_channels
        # For each resolution
        for i in reversed(range(n_resolutions)):
            # `n_blocks` at the same resolution
            out_channels = in_channels
            for _ in range(n_blocks):
                up.append(UpBlock(in_channels, out_channels, n_channels * 4, is_attn[i]))
            # Final block to reduce the number of channels
            out_channels = in_channels // ch_mults[i]
            up.append(UpBlock(in_channels, out_channels, n_channels * 4, is_attn[i]))
            in_channels = out_channels
            # Up sample at all resolutions except last
            if i > 0:
                up.append(Upsample(in_channels))

        # Combine the set of modules
        self.up = nn.ModuleList(up)

        # Final normalization and convolution layer
        self.norm = nn.GroupNorm(8, n_channels)
        self.act = nn.SiLU()
        self.final = nn.Conv2d(in_channels, image_channels, kernel_size=(3, 3), padding=(1, 1))

    def forward(self, x: torch.Tensor, t: torch.Tensor):
        """
        * `x` has shape `[batch_size, in_channels, height, width]`
        * `t` has shape `[batch_size]`
        """

        # Get time-step embeddings
        t = self.time_emb(t)

        # Get image projection
        x = self.image_proj(x)

        # `h` will store outputs at each resolution for skip connection
        h = [x]
        # First half of U-Net
        for m in self.down:
            x = m(x, t)
            h.append(x)

        # Middle (bottom)
        x = self.middle(x, t)

        # Second half of U-Net
        for m in self.up:
            if isinstance(m, Upsample):
                x = m(x, t)
            else:
                # Get the skip connection from first half of U-Net and concatenate
                s = h.pop()
                x = torch.cat((x, s), dim=1)
                #
                x = m(x, t)

        # Final normalization and convolution
        return self.final(self.act(self.norm(x)))

In [4]:
class DiffusionModel(nn.Module):
  def __init__(self,model,steps,s_val,image_shape,channels):
    super(DiffusionModel, self).__init__()
    self.model = model
    self.steps = steps
    self.s_val = s_val
    self.image_shape = image_shape
    self.channels = channels
    self.beta = self.beta_schedule(steps,s_val)
    self.alphas = 1-self.beta
    self.alphas_cumprod = np.cumprod(self.alphas, axis = -1)
  #cosine noise scheduler
  def beta_schedule(self, steps, s_val):
    def f(t):
      return np.cos((t/steps+s_val)/(1+s_val)*0.5*torch.pi)**2
    x = np.linspace(0,steps,steps+1)
    alphas_cumproduct = f(x)/f(np.array([0]))
    beta_ = 1 - alphas_cumproduct[1:]/alphas_cumproduct[:-1]
    beta_ = np.clip(beta_,0.0001,0.999)
    return beta_


  def forward_diffusion(self, image,step):
    try:
      step = step.to('cpu')
    except:
      pass
    sqrt_alphacumprod = torch.Tensor(np.sqrt(self.alphas_cumprod[step])).view(-1,1,1,1).to(device)
    sqrt_oneminus_alphacumprod = torch.Tensor(np.sqrt(1-self.alphas_cumprod[step])).view(-1,1,1,1).to(device)
    epsilon = torch.randn_like(image).to(device)
    noised_image = sqrt_alphacumprod*image.to(device) + sqrt_oneminus_alphacumprod*epsilon
    return noised_image, epsilon

  def backward_diffusion(self, noised_img, step):
    noise = self.model(noised_img, step)
    return noise

  def reconstruct_imgs(self, n_img, time):
    self.model.eval()
    with torch.no_grad():
      denoised_img = n_img.to(device)
      for step in tqdm(range(time)):
        step = time - step - 1
        step = torch.full((1,), step).to(device)
        predicted_noise = self.backward_diffusion(denoised_img,step)
        alpha_t = self.alphas[step]
        cumprod_alpha_t = self.alphas_cumprod[step]

        denoised_img = (1/np.sqrt(alpha_t))*(denoised_img - ((1-alpha_t)/np.sqrt(1 - cumprod_alpha_t))* predicted_noise)
    self.model.train()
    return denoised_img
  def generate_samples(self, number_samples):
    self.model.eval()
    with torch.no_grad():
      samples = torch.randn(number_samples, self.channels, self.image_shape, self.image_shape, device = device)

      for step in tqdm(range(self.steps)):
        step = self.steps - step -1
        step = torch.full((number_samples,), step)
        predicted_noise = self.backward_diffusion(samples, step.to(device))
        alpha_t = self.alphas[step]
        alpha_cumprod_t = self.alphas_cumprod[step]
        samples = (1 / np.sqrt(alpha_t)) * (samples - (1 - alpha_t) / np.sqrt((1 - alpha_cumprod_t)) * predicted_noise)
    self.model.train()
    return samples

##Load in the weights

In [7]:
model = DiffusionModel(UNet().to(device),1000,0.008,32,1)

In [8]:
model.load_state_dict(torch.load('/content/DDPM_weights.pt')) #change directory to the path where DDPM_ood.pt is

<All keys matched successfully>

## OOD detection experiments

In [9]:
#Define in distribution loader and out of distribution loader
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])

id_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
id_loader = DataLoader(id_dataset, batch_size=1, shuffle=True)

ood_dataset = MNIST(root='./data', train=False, download=True, transform=transform)
ood_loader = DataLoader(ood_dataset, batch_size=1, shuffle=True)

100%|██████████| 26421880/26421880 [00:05<00:00, 5203832.37it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 198030.62it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3689437.56it/s]


...

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 4454631.67it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5216464.61it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [10]:
perceptionnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
perceptionnet.eval().to(device);
preprocess_alexnet = transforms.Compose([
    transforms.Resize(224),
    #transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 150MB/s]


In [12]:
criterion = nn.MSELoss()

May take anywhere from 5-7 days to get results for 1000 samples with GPU acceleration.

### In Distribution FashionMNIST

In [ ]:
timesteps = [100,200,400,500,700,999]
results_id = []
for it in range(1000):
    print(it)
    aux = []
    og_image, _ = next(iter(id_loader))
    og_image = og_image.to(device)
    og_image_t = og_image.repeat(1,3,1,1)
    og_image_t = preprocess_alexnet(og_image_t)

    images = og_image.repeat(len(timesteps),1,1,1)
    alexnet_pscore = perceptionnet(og_image_t)[0]
    noised_images, eps = model.forward_diffusion(images, timesteps)
    for idx, img in enumerate(noised_images):

        noised_image = img.unsqueeze(0)
        #noised_image = noised_image.repeat(10,1,1,1)
        recon = model.reconstruct_imgs(noised_image, timesteps[idx])[0]
        loss_mse = criterion(og_image,recon.unsqueeze(0))
        n_img = recon.unsqueeze(0).repeat(1,3,1,1)
        n_img = preprocess_alexnet(n_img)
        recon_pscore = perceptionnet(n_img)[0]
        loss_per = criterion(recon_pscore,alexnet_pscore)
        adict = {'timestep': timesteps[idx],'mse_loss':loss_mse.item(),'perception_loss':loss_per.item(),'sum':loss_mse.item()+loss_per.item()}
        #print(adict)
        aux.append(adict)

    results_id.append(aux)

### Out of distribution MNIST

In [ ]:
timesteps = [100,200,400,500,700,999]
results_ood = []
for it in range(1000):
    print(it)
    aux = []
    og_image, _ = next(iter(ood_loader))
    og_image = og_image.to(device)
    og_image_t = og_image.repeat(1,3,1,1)
    og_image_t = preprocess_alexnet(og_image_t)

    images = og_image.repeat(len(timesteps),1,1,1)
    alexnet_pscore = perceptionnet(og_image_t)[0]
    noised_images, eps = model.forward_diffusion(images, timesteps)
    for idx, img in enumerate(noised_images):

        noised_image = img.unsqueeze(0)
        #noised_image = noised_image.repeat(10,1,1,1)
        recon = model.reconstruct_imgs(noised_image, timesteps[idx])[0]
        loss_mse = criterion(og_image,recon.unsqueeze(0))
        n_img = recon.unsqueeze(0).repeat(1,3,1,1)
        n_img = preprocess_alexnet(n_img)
        recon_pscore = perceptionnet(n_img)[0]
        loss_per = criterion(recon_pscore,alexnet_pscore)
        adict = {'timestep': timesteps[idx],'mse_loss':loss_mse.item(),'perception_loss':loss_per.item(),'sum':loss_mse.item()+loss_per.item()}
        #print(adict)
        aux.append(adict)

    results_ood.append(aux)

## Graphs

In [ ]:
import seaborn as sns

In [ ]:
for j in range(len(results_ood[0])):
  ood_mse_loss100 = []
  for i in results_ood:
    ood_mse_loss100.append(i[j]['sum'])
  id_mse_loss100 = []
  for i in results_id:
    id_mse_loss100.append(i[j]['sum'])
  plt.figure()
  plt.title('T = '+str(results_ood[0][j]['timestep']))
  plt.xlabel('Reconstruction score')
  sns.histplot(data=id_mse_loss100,stat='density',color='blue',label='ID FashionMNIST')
  sns.histplot(data=ood_mse_loss100,stat='density',color='red',label='OOD MNIST')
  plt.legend()
